In [1]:
import pandas as pd
import numpy as np
import time
import datetime
import itertools

In [2]:
# 导入数据集
df_raw = pd.read_excel('./data.xlsx',sheet_name='Sheet4').iloc[:,:4]
# df_raw['日期'] = pd.to_datetime(df_raw['日期'])
# df_raw = df_raw.set_index('日期')
df_raw.head()

,日期,顾客编号,分类,购买量
0,2021-01-01,1,A,4
1,2021-01-02,1,E,1
2,2021-01-06,1,A,4
3,2021-01-10,1,C,4
4,2021-01-10,1,E,3


In [3]:
# 获取间隔日期的最大、最小日期（最小日期往前推 7 日）
date_max = datetime.datetime.strftime(df_raw.日期.max(), '%Y-%m-%d')
date_min = datetime.datetime.strftime(df_raw.日期.min() - datetime.timedelta(days=7), '%Y-%m-%d')
date_max,date_min

('2021-01-23', '2020-12-25')

In [4]:
# 对数据分组
df_group = df_raw.groupby(['顾客编号','分类','日期']).agg('sum').reset_index()
df_group = df_group.sort_values(by=['顾客编号','分类','日期'],ascending=True)
df_group['日期'] = df_group['日期'].apply(lambda x:x.strftime('%Y-%m-%d'))
df_group.head()

,顾客编号,分类,日期,购买量
0,1,A,2021-01-01,4
1,1,A,2021-01-06,4
2,1,A,2021-01-12,5
3,1,B,2021-01-13,3
4,1,C,2021-01-10,4


In [5]:
# 生成连续时间索引
df = pd.DataFrame()
df_index = pd.date_range(date_min,date_max)
df = df.reindex(df_index).reset_index()
df['日期'] = df['index'].apply(lambda x:x.strftime('%Y-%m-%d'))
df.head()

,index,日期
0,2020-12-25,2020-12-25
1,2020-12-26,2020-12-26
2,2020-12-27,2020-12-27
3,2020-12-28,2020-12-28
4,2020-12-29,2020-12-29


In [6]:
# 构建全集
df_custid = list(df_group.顾客编号.unique())
df_catelog = list(df_group.分类.unique())
df_date = list(df['日期'].unique())
temp_list = []
for x in itertools.product(df_date,df_custid,df_catelog):
    temp_list.append(list(x))
temp_df = pd.DataFrame(temp_list)
temp_df.head()

,0,1,2
0,2020-12-25,1,A
1,2020-12-25,1,B
2,2020-12-25,1,C
3,2020-12-25,1,D
4,2020-12-25,1,E


In [7]:
# 左连接
df_merge = pd.merge(temp_df,df_group, how='left', left_on=[1,2,0],right_on=['顾客编号','分类','日期'])
df_merge.drop(columns=['顾客编号','分类','日期'],inplace=True)
df_merge = df_merge.sort_values(by=[1,2,0],ascending=True)
df_merge

,0,1,2,购买量
0,2020-12-25,1,A,NaN
15,2020-12-26,1,A,NaN
30,2020-12-27,1,A,NaN
45,2020-12-28,1,A,NaN
60,2020-12-29,1,A,NaN
...,...,...,...,...
389,2021-01-19,3,E,NaN
404,2021-01-20,3,E,NaN
419,2021-01-21,3,E,NaN
434,2021-01-22,3,E,NaN


In [8]:
# 测试
df = df_merge.assign(rolling_7=df_merge.groupby([1],as_index=False)[['购买量']].rolling(7,min_periods=1).sum().fillna(0).reset_index(0,drop=True))
df

,0,1,2,购买量,rolling_7
0,2020-12-25,1,A,NaN,0.0
15,2020-12-26,1,A,NaN,0.0
30,2020-12-27,1,A,NaN,0.0
45,2020-12-28,1,A,NaN,0.0
60,2020-12-29,1,A,NaN,0.0
...,...,...,...,...,...
389,2021-01-19,3,E,NaN,5.0
404,2021-01-20,3,E,NaN,5.0
419,2021-01-21,3,E,NaN,5.0
434,2021-01-22,3,E,NaN,0.0


In [9]:
# 剔除 购买量 为空的行
df_result = df.dropna(subset=['购买量']).reset_index()
df_result .drop(columns=['index'],inplace=True)
df_result.rename(columns={0:'日期',1:'顾客编号',2:'分类','rolling_7':'近7日购买量'},inplace=True)
df_result 

,日期,顾客编号,分类,购买量,近7日购买量
0,2021-01-01,1,A,4.0,4.0
1,2021-01-06,1,A,4.0,8.0
2,2021-01-12,1,A,5.0,9.0
3,2021-01-13,1,B,3.0,3.0
4,2021-01-10,1,C,4.0,4.0
5,2021-01-12,1,C,3.0,7.0
6,2021-01-13,1,C,3.0,10.0
7,2021-01-15,1,C,4.0,14.0
8,2021-01-14,1,D,3.0,3.0
9,2021-01-02,1,E,1.0,1.0
